In [2]:
import dill as pickle
from pluribus.game.evaluation import Evaluator
import math
import time
import random
import numpy as np
from scipy.stats import wasserstein_distance


In [3]:
with open('data/ehss_river.pkl', 'rb') as file:
    data = pickle.load(file)
with open('data/ehs_dist_turn.pkl', 'rb') as file:
    data2 = pickle.load(file)

In [4]:
# original deck the river centroids were created from
short_deck = data['short_deck']
# speedy look up for printing
eval_to_card = data['eval_to_card']

# river & turn centroids
river_centroids = data['river_centroids']
turn_centroids = data2['turn_centroids']

# combos - I don't think I'll need all of them
on_flop = data['combos'][0]
on_turn = data['combos'][1]
on_river = data['combos'][2]

# expected hand strength on the river.. don't think I'll need it.. ehs distributions on turn
ehss_river = data['ehss_river'] 
ehs_dist_turn = data2['ehs_distributions']
evaluator = Evaluator()

In [5]:
def ncr(n,r):
    """
    helper function for calculating combination size
    n choose r"""
    return int(math.factorial(n)/(math.factorial(r)*math.factorial(n-r)))

In [6]:
print(f'expected flop: {ncr(10,2)*ncr(8,3)}')

expected flop: 2520


In [7]:
print(f'my combos: {len(on_flop)}')

my combos: 2520


In [8]:
# get data for expected hand rates on 3 card public board, not lossless

In [9]:
# making MUCH smaller for quicker time
start = time.time()

# object for EMD on expected of hand strength off flop (distribution)
ehs_distribution_flops = []

for i, public in enumerate(on_flop): 
    available_cards = [x for x in short_deck if x not in public] # this is probably not a good idea
    random.shuffle(available_cards) # can't remember why I did this..
    
    # where i'll store the data for each flop
    ehs_distribution_flop = np.zeros(len(turn_centroids))
    for j in range(7):  
                        # probably want to increase this number?
                        # it's too small maybe for this toy problem

        # randomly generating turn
        turn_card = random.sample(available_cards, 1)

        our_hand = list(public[:2])
        board = public[2:5]
        board = np.append(board, turn_card).tolist()
        
        # getting available cards
        available_cards_turn = [x for x in available_cards if x != turn_card[0]] # oof
        
        
        # the similar object I used in the turn clustering.. get ehs distribution for turn
        ehs_distribution = np.zeros(len(river_centroids))
        # sample river cards and run a simulation
        for k in range(7):
            
            river_card = random.sample(available_cards_turn, 1)
            board = list(public[2:5]) + turn_card # need to define thiss again
            board = board + river_card
            available_cards_river = [x for x in available_cards if x != river_card[0]] # oof
            
            our_hand_rank = evaluator.evaluate(
                board=board, 
                cards=our_hand,
            )
        
            # simulations will be run against randomly drawn opponent hands
            ehs=[0]*3
            for l in range(15): # will want to increase this
                
                # sample from the available cards on the river
                opp_hand = random.sample(available_cards_river, 2)

                opp_hand_rank = evaluator.evaluate(
                        board=board,
                        cards=opp_hand,
                )

                # who wins?
                if our_hand_rank > opp_hand_rank: # maybe some mod magic here
                    idx = 0
                elif our_hand_rank < opp_hand_rank:
                    idx = 1
                elif our_hand_rank == opp_hand_rank:
                    idx = 2

                # increment win rate for winner
                ehs[idx] += 1/15
        
            # get EMD for expected hand strength against each river centroid
            # to which does it belong?
            for idx, river_centroid in enumerate(river_centroids):
                emd = wasserstein_distance(ehs, river_centroid)

                if idx == 0:
                    min_idx = idx
                    min_emd = emd
                else:
                    if emd < min_emd:
                        min_idx = idx
                        min_emd = emd

            # ok, now increment the cluster to which it belongs - 
            ehs_distribution[min_idx] += 1/7 # could also probs be just a regular old integer

        # now same thing for the newly found turn centroids
        for idx, turn_centroid in enumerate(turn_centroids):
            
            #earth mover distance
            emd = wasserstein_distance(ehs_distribution, turn_centroid)

            if idx == 0:
                min_idx = idx
                min_emd = emd
            else:
                if emd < min_emd:
                    min_idx = idx
                    min_emd = emd

        # ok, now increment the cluster to which it belongs - 
        ehs_distribution_flop[min_idx] += 1/7 
        
    # object for storing flop potential aware expected hand strength distributions     
    ehs_distribution_flops.append(ehs_distribution_flop)
          
end = time.time()
print(end - start)

296.69999408721924


In [36]:
len(ehs_distribution_flops)

2520

In [37]:
# simple kmeans algo - should I write from scratch?
from sklearn.cluster import KMeans

X = np.array(ehs_distribution_flops)

km = KMeans(
    n_clusters=15, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)
y_km = km.fit_predict(X)

In [38]:
# centers to be used to get data for EMD
centroids = km.cluster_centers_

In [39]:
data = {
    'ehs_distributions_flops': ehs_distribution_flops,
    'short_deck': short_deck,
    'combos': [on_flop, on_turn, on_river],
    'turn_centroids': centroids,
    'eval_to_card': eval_to_card
}

In [40]:
with open('data/ehs_dist_flop.pkl', 'wb') as file:
    pickle.dump(data, file)

In [41]:
# # to reopen
# with open('data/ehss_river.pkl', 'rb') as file:
#     data = pickle.load(file)

In [42]:
compare_hands = random.sample(list(on_flop[y_km==0]), 5)

In [43]:
# taking a look at them
# I think these are ones with a good chance of drawing, and maybe otherwise what?
for games in compare_hands:
    t = [eval_to_card[x] for x in games.tolist()[:2]]
    print("####Hand")
    print(t)
    t = [eval_to_card[x] for x in games.tolist()[2:]]
    print("####Board")
    print(t)

####Hand
[<Card card=[3 of spades ♠]>, <Card card=[7 of clubs ♣]>]
####Board
[<Card card=[8 of hearts ♥]>, <Card card=[4 of hearts ♥]>, <Card card=[8 of clubs ♣]>]
####Hand
[<Card card=[ace of spades ♠]>, <Card card=[ace of hearts ♥]>]
####Board
[<Card card=[ace of diamonds ♦]>, <Card card=[10 of clubs ♣]>, <Card card=[7 of clubs ♣]>]
####Hand
[<Card card=[ace of spades ♠]>, <Card card=[8 of clubs ♣]>]
####Board
[<Card card=[ace of diamonds ♦]>, <Card card=[ace of hearts ♥]>, <Card card=[10 of clubs ♣]>]
####Hand
[<Card card=[ace of spades ♠]>, <Card card=[10 of clubs ♣]>]
####Board
[<Card card=[ace of diamonds ♦]>, <Card card=[ace of hearts ♥]>, <Card card=[8 of clubs ♣]>]
####Hand
[<Card card=[8 of hearts ♥]>, <Card card=[8 of clubs ♣]>]
####Board
[<Card card=[4 of hearts ♥]>, <Card card=[3 of spades ♠]>, <Card card=[10 of clubs ♣]>]


In [44]:
# ok, this looks understandable
print(np.array(ehs_distribution_flops)[y_km==4][:10])

[[0.71428571 0.         0.         0.28571429 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.42857143 0.         0.         0.57142857 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.42857143 0.         0.14285714 0.28571429 0.         0.
  0.         0.         0.         0.         0.14285714 0.
  0.         0.         0.        ]
 [0.42857143 0.         0.         0.28571429 0.         0.
  0.         0.         0.         0.         0.28571429 0.
  0.         0.         0.        ]
 [0.42857143 0.         0.14285714 0.42857143 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.42857143 0.         0.         0.42857143 0.         0.
  0.         0.14285714 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.42857143 0.14285714 0.14285714 0.28571429 0.         0.
  0.

In [45]:
# ok, this looks understandable
print(np.array(ehs_distribution_flops)[y_km==0][-10:])

[[0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.14285714 0.         0.85714286 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.14285714 0.         0.85714286 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0.

In [5]:
import dill as pickle
with open('data/information_abstraction.pkl', 'rb') as file:
    data2 = pickle.load(file)

In [6]:
data2

{'_cards': [<Card card=[3 of hearts ♥]>,
  <Card card=[7 of diamonds ♦]>,
  <Card card=[3 of spades ♠]>,
  <Card card=[jack of clubs ♣]>,
  <Card card=[9 of hearts ♥]>,
  <Card card=[9 of spades ♠]>,
  <Card card=[6 of spades ♠]>,
  <Card card=[king of hearts ♥]>,
  <Card card=[5 of hearts ♥]>,
  <Card card=[8 of hearts ♥]>],
 '_evals': [139523,
  2114829,
  135427,
  33589533,
  8398611,
  8394515,
  1053707,
  134228773,
  533255,
  4204049],
 '_evals_to_cards': {139523: <Card card=[3 of hearts ♥]>,
  2114829: <Card card=[7 of diamonds ♦]>,
  135427: <Card card=[3 of spades ♠]>,
  33589533: <Card card=[jack of clubs ♣]>,
  8398611: <Card card=[9 of hearts ♥]>,
  8394515: <Card card=[9 of spades ♠]>,
  1053707: <Card card=[6 of spades ♠]>,
  134228773: <Card card=[king of hearts ♥]>,
  533255: <Card card=[5 of hearts ♥]>,
  4204049: <Card card=[8 of hearts ♥]>},
 'starting_hands': array([[   139523,   2114829],
        [   139523,    135427],
        [   139523,  33589533],
        [ 

In [8]:
data2['_flop_potential_aware_distributions'][-10:]

array([[0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0. , 0. , 0.6, 0. ,
        0. , 0. ],
       [0. , 0. , 0.6, 0. , 0. , 0. , 0. , 0. , 0.2, 0. , 0. , 0.2, 0. ,
        0. , 0. ],
       [0. , 0. , 0.6, 0. , 0. , 0. , 0. , 0. , 0.2, 0. , 0. , 0.2, 0. ,
        0. , 0. ],
       [0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.8, 0. ,
        0. , 0. ],
       [0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0. ,
        0.6, 0. ],
       [0. , 0. , 0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0. ,
        0. , 0. ],
       [0. , 0. , 0.8, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0. ,
        0. , 0. ],
       [0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0.2, 0. , 0. , 0.4, 0. ,
        0.2, 0. ],
       [0. , 0. , 0.8, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0. ,
        0. , 0. ],
       [0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0.2, 0. , 0. , 0.6, 0. ,
        0. , 0. ]])

In [12]:
print(data2['_flop_potential_aware_distributions'][data2['_flop_clusters'] == 2])

[[0.2 0.  0.  0.  0.  0.  0.  0.  0.6 0.  0.  0.  0.  0.2 0. ]
 [0.2 0.  0.  0.  0.  0.  0.  0.  0.4 0.  0.  0.  0.  0.4 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.4 0.  0.  0.  0.2 0.4 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.6 0.  0.  0.2 0.  0.2 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.6 0.  0.  0.  0.  0.4 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.4 0.  0.  0.  0.2 0.4 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.6 0.  0.  0.  0.  0.4 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.6 0.  0.  0.  0.  0.4 0. ]
 [0.2 0.  0.2 0.  0.  0.  0.  0.  0.6 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.4 0.  0.  0.  0.2 0.4 0. ]
 [0.2 0.  0.  0.  0.  0.  0.  0.  0.4 0.  0.  0.2 0.  0.2 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.4 0.  0.  0.2 0.2 0.2 0. ]
 [0.  0.  0.2 0.  0.  0.  0.  0.  0.4 0.  0.  0.2 0.  0.2 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.4 0.  0.  0.2 0.2 0.2 0. ]
 [0.  0.  0.2 0.  0.  0.  0.  0.  0.6 0.  0.  0.  0.  0.2 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.6 0.  0.  0.  0.2 0

In [10]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [15]:
print(data2['_turn_ehs_distributions'][data2['_turn_clusters'] == 10])

[[0.  0.6 0.  0.  0.  0.  0.  0.  0.4 0.  0.  0.  0.  0.  0. ]
 [0.  0.6 0.  0.  0.  0.2 0.  0.  0.2 0.  0.  0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.2 0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0. ]
 [0.  0.6 0.  0.  0.  0.  0.  0.  0.2 0.  0.2 0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0. ]
 [0.  0.6 0.  0.  0.  0.2 0.  0.  0.2 0.  0.  0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0. ]
 [0.  0.6 0.  0.  0.  0.2 0.  0.  0.2 0.  0.  0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0. ]
 [0.  0.6 0.  0.  0.  0.2 0.  0.  0.2 0.  0.  0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0